In [1]:
import pandas as pd

In [2]:
mat = pd.read_csv("student-mat.csv", sep=";")
por = pd.read_csv("student-por.csv", sep=";")

In [3]:
mat.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [4]:
girls, boys = mat['sex'].value_counts();

In [5]:
students_17_year_old = mat.loc[(mat['age'] == 17) & (mat['G3'] >= 10) & (mat['G3'] <= 15)].value_counts().size;

In [6]:
mat['avg'] = mat.apply(
    lambda row: (row['G1'] + row['G2'] + row['G3']) / 3,
    axis=1
)

boys_from_top_10 = mat.nlargest(10,'avg').loc[mat['sex'] == 'M'].value_counts().size

mat = mat.drop('avg', 1)

In [7]:
romantic_internet = mat.loc[(mat['romantic'] == 'yes') & (mat['internet'] == 'yes')]
mean, std = romantic_internet['absences'].mean(), romantic_internet['absences'].std()
print(mean, std);

8.155172413793103 11.25329203873895


In [8]:
student_counts_for_each_age = mat['age'].value_counts().sort_index().tolist()
print(student_counts_for_each_age);

[82, 104, 98, 82, 24, 3, 1, 1]


In [9]:
father, mother = mat.loc[(mat['Fjob'] != 'other') & (mat['Mjob'] != 'other')].groupby(['Fjob','Mjob']).size().reset_index().sort_values(by = 0, ascending = False).iloc[0].drop(0)
print(father)
print(mother);

services
services


In [10]:
columns = list(filter(lambda elem: (elem!='G1') & (elem!='G2') & (elem!='G3') & (elem!='avg'),mat.columns.tolist()))
print(columns)
both_present = pd.merge(mat,por, how='inner', on=columns).value_counts().size
print(both_present);

['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
39


In [11]:
GP_por = por.loc[por['school'] == 'GP']
GP_mat = mat.loc[mat['school'] == 'GP']
GP_diff = abs(GP_por['G3'].mean() - GP_mat['G3'].mean())
print(GP_diff)
MS_por = por.loc[por['school'] == 'MS']
MS_mat = mat.loc[mat['school'] == 'MS']
MS_diff = abs(MS_por['G3'].mean() - MS_mat['G3'].mean())
print(MS_diff);

2.0868608045953643
0.8026163909195851


In [12]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
new_mat = mat.select_dtypes(include=numerics)
mat_cor = new_mat.corr().abs().unstack().reset_index().sort_values(by = 0, ascending = False)
mat_cor = mat_cor.loc[mat_cor['level_0'] != mat_cor['level_1']]
mat_cor['pair'] = mat_cor.apply(
    lambda row: sorted([row['level_0'], row['level_1']]).__str__().replace('[','').replace(']',''),
    axis = 1
)
mat_cor = mat_cor.drop('level_0', 1).drop('level_1', 1).drop_duplicates('pair')
top5_correlations = list(map(lambda elem: elem.replace(' ', '').replace('\'','').split(','), mat_cor.nlargest(5,0)['pair'].tolist()))
print(top5_correlations);

[['G2', 'G3'], ['G1', 'G2'], ['G1', 'G3'], ['Dalc', 'Walc'], ['Fedu', 'Medu']]


In [13]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{boys} {girls}\n")
    file.write(f"{students_17_year_old}\n")
    file.write(f"{boys_from_top_10}\n")
    file.write(f"{round(mean, 3)} {round(std, 3)}\n")
    file.write(f"{student_counts_for_each_age}\n")
    file.write(f"{father} {mother}\n")
    file.write(f"{both_present}\n")
    file.write(f"{round(GP_diff, 3)} {round(MS_diff, 3)}\n")
    file.write(f"{top5_correlations}")